In [50]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

In [48]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id='0c78fdc53b5d482992b86a7f00fc434f', client_secret='924ef3f4065d47ffbd22dfe4939c8625')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [51]:
#Top 50 Playlists we want to include in the dataset
playlists_dic = {'UK': 'https://open.spotify.com/playlist/37i9dQZEVXbLnolsZ8PSNw?si=8f637c1a08824ec0',
                'Global': 'https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=a7d5cae2d67540be',
                'USA': 'https://open.spotify.com/playlist/37i9dQZEVXbLRQDuF5jeBp?si=d7a363c922d24c8a',
                'Germany': 'https://open.spotify.com/playlist/37i9dQZEVXbJiZcmkrIHGU?si=514023410612425e',
                'Japan': 'https://open.spotify.com/playlist/37i9dQZEVXbKXQ4mDTEBXq?si=46ca3e2a5c5a4b7b',
                'France': 'https://open.spotify.com/playlist/37i9dQZEVXbIPWwFssbupI?si=b7a0c888911d461f',
                'Italy': 'https://open.spotify.com/playlist/37i9dQZEVXbIQnj7RRhdSX?si=0108ca7bd805432b',
                'Turkey': 'https://open.spotify.com/playlist/37i9dQZEVXbIVYVBNw9D5K?si=472f47e033574f87',
                'Saudi Arabia': 'https://open.spotify.com/playlist/37i9dQZEVXbLrQBcXqUtaC?si=f1321bfaaeda4a46',
                'Brazil': 'https://open.spotify.com/playlist/37i9dQZEVXbMXbN3EUUhlg?si=2519da35b8504821',
                'Nigeria': 'https://open.spotify.com/playlist/37i9dQZEVXbKY7jLzlJ11V?si=e7c9fe08832c45b9',
                'Columbia':'https://open.spotify.com/playlist/37i9dQZEVXbOa2lmxNORXQ?si=6dc7980b002a4800',
                'Mexico': 'https://open.spotify.com/playlist/37i9dQZEVXbO3qyFxbkOE1?si=960c253fe16d4bf8',
                'Greece': 'https://open.spotify.com/playlist/37i9dQZEVXbJqdarpmTJDL?si=b356536d01e743ed',
                'South Korea': 'https://open.spotify.com/playlist/37i9dQZEVXbNxXF4SkHj9F?si=8bfaf89e9adf4729',
                'Hong Kong': 'https://open.spotify.com/playlist/37i9dQZEVXbLwpL8TjsxOG?si=1316a72c3bbb40b3',
                'Sweden': 'https://open.spotify.com/playlist/37i9dQZEVXbLoATJ81JYXz?si=392b44eed3ea420d',
                'South Africa': 'https://open.spotify.com/playlist/37i9dQZEVXbMH2jvi6jvjk?si=49fca795acc54e87',
                'Spain':'https://open.spotify.com/playlist/37i9dQZEVXbNFJfN1Vw8d9?si=963aa5a7a3b84399',
                'Australia': 'https://open.spotify.com/playlist/37i9dQZEVXbJPcfkRz0wJ0?si=535dae3818d84115',
                'Austria': 'https://open.spotify.com/playlist/37i9dQZEVXbKNHh6NIXu36?si=74842cff3af44134',
                'Poland': 'https://open.spotify.com/playlist/37i9dQZEVXbN6itCcaL3Tt?si=022f4f88429246e8',
                'Ukraine': 'https://open.spotify.com/playlist/37i9dQZEVXbKkidEfWYRuD?si=faae0890feea4be2',
                'Switzerland': 'https://open.spotify.com/playlist/37i9dQZEVXbJiyhoAPEfMK?si=b4196af8525b4de7'}

#Function returns a list of dictionaries each corresponding to a track for one playlist
def get_playlist_tracks(playlist_URI, playlist_name):
    results = sp.playlist_tracks(playlist_URI)
    tracks_data = []
    for track in results['items']:
        audio_features = sp.audio_features(track['track']['id'])[0]
        artist_uri = track['track']['artists'][0]['uri']
        artist = sp.artist(artist_uri)
        track_data = {
            'Track_Name': track['track']['name'],
            'Artist': ', '.join([artist['name'] for artist in track['track']['artists']]),
            'Genres': track['track']['artists'],
            'Release Date': track['track']['album']['release_date'],
            'duration_ms': track['track']['duration_ms'],
            'track_popularity': track['track']['popularity'],
            'explicit': track['track']['explicit'],
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'key': audio_features['key'],
            'loudness': audio_features['loudness'],
            'mode': audio_features['mode'],
            'speechiness': audio_features['speechiness'],
            'acousticness': audio_features['acousticness'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'valence': audio_features['valence'],
            'tempo': audio_features['tempo'],
            'time_signature': audio_features['time_signature'],
            'artist_genres': artist['genres'],
            'artist_followers': artist['followers']['total'],
            'artist_popularity': artist['popularity'],
            'Country': playlist_name
        }
        tracks_data.append(track_data)
    return tracks_data


In [46]:
#Loop over playlists to make dataset
data = []
for country, link in playlists_dic.items():
    uri = link.split("/")[-1].split("?")[0]
    playlist_data = get_playlist_tracks(uri, country)
    data += playlist_data

data = pd.DataFrame(data)
data

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=6t18CcJVpnFAoWfcZxiZtk:
 Max Retries, reason: too many 429 error responses